In [164]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [165]:
df = pd.read_csv("diabetes.csv")

In [166]:
df.fillna(df.mean(), inplace=True)

In [167]:
X = df.drop(columns='Outcome')
y = df['Outcome']

In [168]:
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [169]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=1)

In [170]:
xtrain = torch.tensor(xtrain, dtype=torch.float32)
xtest = torch.tensor(xtest, dtype=torch.float32)
ytrain =torch.tensor(ytrain.values, dtype=torch.float32).unsqueeze(1)
ytest = torch.tensor(ytest.values, dtype=torch.float32).unsqueeze(1)

In [171]:
class Diabetesclassifier(nn.Module):
    def __init__(self):
        super( Diabetesclassifier, self).__init__()
        self.layer_1 = nn.Linear(xtrain.shape[1], 16)
        self.layer_2 = nn.Linear(16, 8)
        self.layer_out = nn.Linear(8,1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.relu(self.layer_2(x))
        x = self.sigmoid(self.layer_out(x))
        return x

In [172]:
model = Diabetesclassifier()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [173]:

epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(xtrain)
    loss = criterion(outputs, ytrain)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
        
              

Epoch [10/500], Loss: 0.6533
Epoch [20/500], Loss: 0.6440
Epoch [30/500], Loss: 0.6350
Epoch [40/500], Loss: 0.6259
Epoch [50/500], Loss: 0.6161
Epoch [60/500], Loss: 0.6054
Epoch [70/500], Loss: 0.5937
Epoch [80/500], Loss: 0.5810
Epoch [90/500], Loss: 0.5673
Epoch [100/500], Loss: 0.5528
Epoch [110/500], Loss: 0.5385
Epoch [120/500], Loss: 0.5245
Epoch [130/500], Loss: 0.5113
Epoch [140/500], Loss: 0.4996
Epoch [150/500], Loss: 0.4892
Epoch [160/500], Loss: 0.4803
Epoch [170/500], Loss: 0.4726
Epoch [180/500], Loss: 0.4663
Epoch [190/500], Loss: 0.4611
Epoch [200/500], Loss: 0.4569
Epoch [210/500], Loss: 0.4534
Epoch [220/500], Loss: 0.4503
Epoch [230/500], Loss: 0.4476
Epoch [240/500], Loss: 0.4451
Epoch [250/500], Loss: 0.4430
Epoch [260/500], Loss: 0.4411
Epoch [270/500], Loss: 0.4392
Epoch [280/500], Loss: 0.4374
Epoch [290/500], Loss: 0.4357
Epoch [300/500], Loss: 0.4339
Epoch [310/500], Loss: 0.4323
Epoch [320/500], Loss: 0.4308
Epoch [330/500], Loss: 0.4292
Epoch [340/500], Lo

In [174]:
model.eval()
with torch.no_grad():
    ypred_train = model(xtrain)
    ypred_test = model(xtest)
    
    ypred_train_cls = (ypred_train >= 0.5).float()
    ypred_test_cls = (ypred_test >= 0.5).float()

    # Ensure that ytest and ypred_test_cls have the same shape
    #print(f'ytest shape: {ytest.shape}')
    #print(f'ypred_test_cls shape: {ypred_test_cls.shape}')

    accuracy = accuracy_score(ytest, ypred_test_cls)
    mse = mean_squared_error(ytest, ypred_test)

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Mean Squared Error: {mse:.4f}')

Accuracy: 0.8281
Mean Squared Error: 0.1265
